In [ ]:
# =============================================================================
# Authors: Deepthi Kuttichira and Pubudu Sanjeewani Thihagoda Gamage
# Date: 20-08-2024
# Title: A Novel Non-Iterative Training Method for CNN Classifiers Using Gram-Schmidt Process
# License: MIT License
# =============================================================================

# Import necessary libraries
import numpy as np
from tensorflow.keras.datasets import cifar10
from sklearn.model_selection import train_test_split

# Load CIFAR-10 dataset and print shapes
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print((x_train.shape,y_train.shape)) 
print((x_test.shape,y_test.shape)) 

# Load precomputed feature matrix and remove zero-only columns (Change the file path to the actual location on the system)
A = np.load(r'C:\Users\sanje\Desktop\Griffith\code\pretrained\cifar10train_2000relu_vgg16.npy')
A = A[:, np.any(A != 0, axis=0)] 

# Use training labels as target vector
b = y_train
print(f"Shape of feature matrix A: {A.shape}")
print(f"Shape of label vector b: {b.shape}")

# Gram-Schmidt QR factorization function
def gram_schmidt_qr(A):
    m, n = A.shape
    Q = np.zeros((m, n))
    R = np.zeros((n, n))
    for j in range(n):
        v = A[:, j]
        for i in range(j):
            q = Q[:, i]
            R[i, j] = np.dot(q, v)
            v = v - R[i, j] * q
        R[j, j] = np.linalg.norm(v)
        Q[:, j] = v / R[j, j]
    return Q, R

# Back-substitution solver for Rx = c
def back_substitution(R, c):
    n = len(c)
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = c[i]
        for j in range(i+1, n):
            x[i] -= R[i, j] * x[j]
        x[i] /= R[i, i]
    return x

# Perform one-vs-all classification using QR for each class
unique_classes = np.unique(b)
print("Unique Classes")
print(unique_classes)
weights = []

for cls in unique_classes:
    print(cls)
    b_current_class = np.where(b == cls, 1, 0) # Binary target for current class
    Q, R = gram_schmidt_qr(A)                  # QR factorization
    c = np.dot(Q.T, b_current_class)           # Compute c = Qᵀb
    x = back_substitution(R, c)                # Solve for weights
    weights.append(x)                          # Store weights

# Save computed weights to file
print("Weights for each class:", weights)
w=np.array(weights)
np.save("qrweightcifar10_2000relu_vgg16.npy",w)

((50000, 32, 32, 3), (50000, 1))
((10000, 32, 32, 3), (10000, 1))
Shape of feature matrix A: (50000, 988)
Shape of label vector b: (50000, 1)
Unique Classes
[0 1 2 3 4 5 6 7 8 9]
0
1
2
3
4
5
6
7
8
9
Weights for each class: [array([ 1.90585009e-02, -9.00554710e-02, -9.24656886e-03,  1.01797515e-01,
       -1.64542566e-02, -9.78646320e-02,  1.46404357e-02, -6.07472871e-02,
       -7.04284797e-02,  1.12979430e-01, -3.18367621e-03, -7.83278730e-02,
        1.80591957e-02,  7.88925442e-03, -8.75885546e-02,  2.17878356e-03,
       -2.25532835e-02, -7.61356108e-02, -3.88935384e-02, -2.70290762e-02,
        5.56015405e-02, -6.93962379e-02, -2.17162788e-02,  2.32820683e-01,
        6.08865612e-02,  3.03918277e-02,  3.72872805e-03,  1.24885122e-01,
        1.15897911e-01, -4.01253792e-02, -4.71182718e-02,  1.53130855e-02,
        3.33353611e-02,  2.43416329e-02, -6.84487332e-02, -3.94859362e-02,
        3.73650646e-02, -9.02691349e-03, -2.49767261e-02,  2.12069690e-01,
       -5.28748842e-02,  4